## Connection to the mongodb driver. 

The connector can be used to access remote database (e.g. Mongo Atlas) as well.

**N.B.** You should install the Python package (*pip install pymongo*) to use the API.

In [1]:
import pymongo
import re 


# Connection
client = pymongo.MongoClient('localhost', 27017)

## Yelp Dataset

### Load Yelp Dataset

In [2]:
import json 

path = "C://Users//annal//Documents//GitHub//BigDataHomework//Homework1//yelp dataset//"
business = []

with open(path + 'yelp_academic_dataset_business.json', 'rb') as f:
    for jsonObj in f:
        business_dict = json.loads(jsonObj)
        business.append(business_dict)


In [3]:
review = []
count=0

with open(path + 'yelp_academic_dataset_review.json', 'rb') as f:
    for jsonObj in f:
        review_dict = json.loads(jsonObj)
        review.append(review_dict)

In [13]:
user = []
count = 0

with open(path + 'yelp_academic_dataset_user.json', 'rb') as f:
    for jsonObj in f:
        user_dict = json.loads(jsonObj)
        user.append(user_dict)
              

In [ ]:
checkin = []

with open(path + 'yelp_academic_dataset_checkin.json', 'rb') as f:
    for jsonObj in f:
        checkin_dict = json.loads(jsonObj)
        checkin.append(checkin_dict)


In [ ]:
tip = []

with open(path + 'yelp_academic_dataset_tip.json', 'rb') as f:
    for jsonObj in f:
        tip_dict = json.loads(jsonObj)
        tip.append(tip_dict)

## Preprocessing

## Prendo i Ristoranti

In [4]:
restaurants=[]
temp=""
for i in range(0,len(business)):
    business_id=business[i]['business_id']
    stringa=str(business[i]['categories'])
    chunks=stringa.split(',')
    if ('Restaurants' in chunks):
        restaurants.append(business[i])       

In [5]:
allBusinessId=[]
for r in restaurants:
    allBusinessId.append(r['business_id'])

In [6]:
reviewRestaurants=[]

for r in review[:1000]:
    if (r['business_id'] in allBusinessId):
        reviewRestaurants.append(r)

### Mi collego al db

In [7]:
import pymongo

# Connection
client = pymongo.MongoClient('localhost', 27017)

# Select the db by its name
db = client['yelpPython']

### Carico le collection 

In [8]:
#Cancello tutte le collezioni all'inizio per evitare sovrascrizioni

lista_collection=list(db.list_collection_names())

for col in lista_collection:
    mycol = db[col]
    mycol.drop()


In [9]:
# print out available databases names
print('Available database -> ', client.list_database_names())
print('Available collections -> ', db.list_collection_names())

Available database ->  ['Yelp', 'admin', 'config', 'local', 'students', 'twitter']
Available collections ->  []


In [45]:
# Create a collection
#collection_business = db['yelpBusiness']
#res = collection_business.insert_many(business)

In [10]:
# Create a collection for the restaurants
collection_business_restaurants = db['yelpBusinessRestaurants']
res = collection_business_restaurants.insert_many(restaurants)

In [11]:
# Create a collection for restaurants
collection_review_restaurants = db['yelpReviewRestaurants']
res = collection_review_restaurants.insert_many(reviewRestaurants)

In [14]:
# Create a collection
collection_user= db['yelpUser']
res = collection_user.insert_many(user)

In [11]:
# Create a collection
collection_checkin = db['yelpCheckin']
res = collection_checkin.insert_many(checkin)

In [12]:
# Create a collection
collection_tip= db['yelpTip']
res = collection_tip.insert_many(tip)

In [198]:
collection_review = db['yelpReview']
res = collection_review.insert_many(review[:10000])

In [12]:
# print out available databases names
print('Available database -> ', client.list_database_names())
print('Available collections -> ', db.list_collection_names())

Available database ->  ['Yelp', 'admin', 'config', 'local', 'students', 'twitter', 'yelpPython']
Available collections ->  ['yelpReviewRestaurants', 'yelpBusinessRestaurants']


### Query Business

In [16]:
#Numero di recensioni a partire dal nome del business
filter={
    'name': 'St Honore Pastries', 
}

project={
    'name': 1, 
    'review_count': 1,
    'stars':1
}

result = collection_business_restaurants.find(
    filter=filter,
    projection=project
)


result_list = list(result)
result_list

[{'_id': ObjectId('626ac309d6d9f55434ea0178'),
  'name': 'St Honore Pastries',
  'stars': 4.0,
  'review_count': 80}]

In [37]:
reviewRestaurants[2]

{'review_id': 'UBp0zWyH60Hmw6Fsasei7w',
 'user_id': '4Uh27DgGzsp6PqrH913giQ',
 'business_id': 'otQS34_MymijPTdNBoBdCw',
 'stars': 4.0,
 'useful': 0,
 'funny': 2,
 'cool': 0,
 'text': "The bun makes the Sonoran Dog. It's like a snuggie for the pup. A first, it seems ridiculous and almost like it's going to be too much, exactly like everyone's favorite blanket with sleeves. Too much softness, too much smush, too indulgent.  Wrong. It's warm, soft, chewy, fragrant, and it succeeds where other famed Sonoran Dogs fail. \n\nThe hot dog itself is flavorful, but I would prefer that it or the bacon have a little more bite or snap to better hold their own against the dominant mustard and onions. \n\nI'm with the masses on the carne asada caramelo. Excellent tortilla, salty, melty cheese, and great carne. \n\nSuper cheap and you can drive through.",
 'date': '2011-10-27 17:12:05',
 '_id': ObjectId('626a78d25fdc03c1ca1cab27')}

In [17]:
for i in range(0,len(restaurants)):
    if restaurants[i]['business_id']=='otQS34_MymijPTdNBoBdCw':
        print(restaurants[i]['name'])

BK Tacos


In [18]:
#Dato il Nome di un Negozio vedere le recensioni
pipe = [
    {
        '$lookup':{
            'from': "yelpBusinessRestaurants",
            'localField': "business_id",
            'foreignField': "business_id",
            'as': "yelpBusiness"
            }

    }, {
        '$unwind': "$yelpBusiness"
    },
    {
        '$match': {
            "yelpBusiness.name":{'$eq':"BK Tacos"}
            }
    },
    {
        '$project': {
            'yelpBusiness.name': 1,
            'text': 1,
            'stars':1
        }
    }

]

# filtering
result = collection_review_restaurants.aggregate(pipe)
risultato=list(result)
risultato

[{'_id': ObjectId('626ac333d6d9f55434ea3d34'),
  'stars': 4.0,
  'text': "The bun makes the Sonoran Dog. It's like a snuggie for the pup. A first, it seems ridiculous and almost like it's going to be too much, exactly like everyone's favorite blanket with sleeves. Too much softness, too much smush, too indulgent.  Wrong. It's warm, soft, chewy, fragrant, and it succeeds where other famed Sonoran Dogs fail. \n\nThe hot dog itself is flavorful, but I would prefer that it or the bacon have a little more bite or snap to better hold their own against the dominant mustard and onions. \n\nI'm with the masses on the carne asada caramelo. Excellent tortilla, salty, melty cheese, and great carne. \n\nSuper cheap and you can drive through.",
  'yelpBusiness': {'name': 'BK Tacos'}},
 {'_id': ObjectId('626ac333d6d9f55434ea3d76'),
  'stars': 4.0,
  'text': "I was told this place is a must for a Sonoran hot dog. I was visiting from out of town and had never had one. It was good, but the whole Sonoran

### Query Cliente

In [19]:
# Trova i ristoranti aperti con più di 4 stelle ed ordinali in modo decrescente
pipe=[
    {
        '$match':  {
            'stars': {
                '$gt': 4.0
            },
            'is_open':{'$eq':1}
        }
    },
    {
        '$sort' : { 'stars' : -1, 'city': 1 } 
    },
    {
        '$project': {
            '_id': 0,
            'name': 1,
            'city':1,
            'stars':1,
            'is_open':1
            
        }
    }
]

result = collection_business_restaurants.aggregate(pipe)

result_list = list(result)
len(result_list)

1855

In [77]:
result_list[:10]

[{'name': '2 Fat Dogs', 'city': 'Abington', 'stars': 5.0, 'is_open': 1},
 {'name': "Long John Silver's", 'city': 'Arnold', 'stars': 5.0, 'is_open': 1},
 {'name': 'Q Tea Vietnamese Cafe',
  'city': 'Arnold',
  'stars': 5.0,
  'is_open': 1},
 {'name': "Ryan's Country Store", 'city': 'Aston', 'stars': 5.0, 'is_open': 1},
 {'name': 'Desserts by Design', 'city': 'Audubon', 'stars': 5.0, 'is_open': 1},
 {'name': 'Core de Roma', 'city': 'Bala Cynwyd', 'stars': 5.0, 'is_open': 1},
 {'name': 'Rosalia’s Pizza', 'city': 'Berlin', 'stars': 5.0, 'is_open': 1},
 {'name': 'Graveyard B&G', 'city': 'Boise', 'stars': 5.0, 'is_open': 1},
 {'name': 'ā café', 'city': 'Boise', 'stars': 5.0, 'is_open': 1},
 {'name': 'Off the Grid Pizza', 'city': 'Boise', 'stars': 5.0, 'is_open': 1}]

In [79]:
restaurants[1]

{'business_id': '9OG5YkX1g2GReZM0AskizA',
 'name': "Romano's Macaroni Grill",
 'address': '5505 S Virginia St',
 'city': 'Reno',
 'state': 'NV',
 'postal_code': '89502',
 'latitude': 39.4761165,
 'longitude': -119.7893392,
 'stars': 2.5,
 'review_count': 339,
 'is_open': 1,
 'attributes': {'RestaurantsGoodForGroups': 'True',
  'RestaurantsTakeOut': 'True',
  'RestaurantsAttire': "'casual'",
  'NoiseLevel': "u'average'",
  'OutdoorSeating': 'True',
  'BikeParking': 'True',
  'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
  'RestaurantsPriceRange2': '2',
  'Caters': 'True',
  'GoodForKids': 'True',
  'RestaurantsReservations': 'True',
  'BusinessAcceptsCreditCards': 'True',
  'Alcohol': "'full_bar'",
  'GoodForMeal': "{'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False}",
  'RestaurantsTableService': 'True',
  'HappyHour': 'True',
  'WiFi': "u'free'",
  'DogsAllowed': 'False',
 

In [20]:
#trova i ristoranti aperti in una certa città ordinandoli per stelle
filter={
    'city': 'Philadelphia', 
    'is_open': 1
}

project={
    '_id':0,
    'name': 1, 
    'is_open':1,
    'address':1,
    'stars':1,
    'attributes':1,
    'hours':1
}

result = collection_business_restaurants.find(
    filter=filter,
    projection=project
).sort('stars', -1)


result_list = list(result)
result_list[:2]

[{'name': 'El Guero  Mexican Food Truck',
  'address': '1256 W Montgomery Ave',
  'stars': 5.0,
  'is_open': 1,
  'attributes': {'RestaurantsTakeOut': 'True',
   'OutdoorSeating': 'False',
   'RestaurantsTableService': 'False',
   'Caters': 'True',
   'HasTV': 'False',
   'RestaurantsDelivery': 'True',
   'Alcohol': "u'none'",
   'WiFi': "u'no'",
   'WheelchairAccessible': 'True',
   'BusinessAcceptsCreditCards': 'True',
   'BikeParking': 'True'},
  'hours': {'Monday': '10:0-18:0',
   'Tuesday': '10:0-18:0',
   'Wednesday': '10:0-18:0',
   'Thursday': '10:0-18:0',
   'Friday': '10:0-18:0'}},
 {'name': 'Jade Palace',
  'address': '1714 S 5th St',
  'stars': 5.0,
  'is_open': 1,
  'attributes': {'HasTV': 'True', 'RestaurantsDelivery': 'True'},
  'hours': {'Monday': '11:0-21:30',
   'Tuesday': '11:0-21:30',
   'Wednesday': '11:0-21:30',
   'Thursday': '11:0-21:30',
   'Friday': '11:0-21:30',
   'Saturday': '11:0-21:30',
   'Sunday': '11:0-21:0'}}]

### Query Yelp

In [142]:
#Le prime 10 città che hanno più ristoranti
pipe=[
    {
        '$group':{
            '_id': '$city', 'Totale':{ '$count': {}}
        }
    },
    {
        '$sort': {
            'Totale':-1
        }
    },
    {
        '$limit' : 10
    }
]

result=collection_business_restaurants.aggregate(pipe)
res = list(result)
res

[{'_id': 'Philadelphia', 'Totale': 1730},
 {'_id': 'Tampa', 'Totale': 848},
 {'_id': 'Indianapolis', 'Totale': 827},
 {'_id': 'Tucson', 'Totale': 742},
 {'_id': 'Edmonton', 'Totale': 693},
 {'_id': 'Nashville', 'Totale': 660},
 {'_id': 'New Orleans', 'Totale': 602},
 {'_id': 'Saint Louis', 'Totale': 528},
 {'_id': 'Reno', 'Totale': 366},
 {'_id': 'Boise', 'Totale': 230}]

### Retrieve top-k result

In [183]:
# Ristoranti aperti con il maggior numero di recensioni
filter={
   'is_open': {
        '$ne': 0
    }
}

# data projection
project={
    'name':1,
    'review_count': 1,
     'stars': 1,
    '_id': 0
}

sort=list({
    'review_count': -1,
    'stars':-1
}.items())

limit = 100

result = collection_business_restaurants.find(
    filter=filter,
    projection=project,
    sort = sort,
    limit = limit
)

result_list = list(result)
result_list

[{'name': 'Oceana Grill', 'stars': 4.0, 'review_count': 7400},
 {'name': 'Ruby Slipper - New Orleans', 'stars': 4.5, 'review_count': 5193},
 {'name': "Felix's Restaurant & Oyster Bar",
  'stars': 4.0,
  'review_count': 3966},
 {'name': "Drago's Seafood Restaurant", 'stars': 3.5, 'review_count': 3160},
 {'name': 'Parc', 'stars': 4.0, 'review_count': 2761},
 {'name': 'Green Eggs Café', 'stars': 4.0, 'review_count': 2679},
 {'name': "Peg's Glorified Ham N Eggs", 'stars': 4.5, 'review_count': 2506},
 {'name': "Deanie's Seafood", 'stars': 3.5, 'review_count': 2473},
 {'name': "Tommy DiNic's", 'stars': 4.0, 'review_count': 2221},
 {'name': "Martin's Bar-B-Que Joint", 'stars': 4.0, 'review_count': 2221},
 {'name': 'Parkway Bakery & Tavern', 'stars': 4.5, 'review_count': 2160},
 {'name': 'Prep & Pastry', 'stars': 4.5, 'review_count': 2126},
 {'name': 'The Pancake Pantry', 'stars': 4.0, 'review_count': 2063},
 {'name': 'Bourbon House', 'stars': 3.5, 'review_count': 2026},
 {'name': 'Domenica', 

### Sentimental Analysis

In [210]:

import re 

filter={
    '$or' : [
        { 'text': {'$regex': re.compile(r"good")}},
        { 'text': {'$regex': re.compile(r"amazing")}},
        { 'text': {'$regex': re.compile(r"great")}},
        { 'text': {'$regex': re.compile(r"cool")}}
    ]   
}

project={
    'name': 1,
    'stars': 1, 
    'text':1,
    '_id':0
}


result = collection_review.find(
  filter=filter,
  projection=project
)

result_list = list(result)
len(result_list),result_list[0]

(5764,
 {'stars': 3.0,
  'text': "If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker."})

## Indices

In [51]:
from pymongo import ASCENDING, DESCENDING
db.collection_business_restaurants.create_index([ ("city", ASCENDING) ])


'city_1'

In [54]:
print(list(db.collection_business_restaurants.list_indexes()))

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')]), SON([('v', 2), ('key', SON([('city', 1)])), ('name', 'city_1')])]


In [48]:
db.collection_business_restaurants.drop_indexes()

In [53]:
db.collection_business_restaurants.find({ 'city': 'Santa Barbara' })

In [52]:
db.collection_business_restaurants.find({ 'city': 'Santa Barbara' })

In [45]:
ris=db.collection_business_restaurants.find({ 'city': 'Santa Barbara' }).explain()


In [46]:
print(ris)

{'explainVersion': '1', 'queryPlanner': {'namespace': 'yelpPython.collection_business_restaurants', 'indexFilterSet': False, 'parsedQuery': {'city': {'$eq': 'Santa Barbara'}}, 'maxIndexedOrSolutionsReached': False, 'maxIndexedAndSolutionsReached': False, 'maxScansToExplodeReached': False, 'winningPlan': {'stage': 'FETCH', 'inputStage': {'stage': 'IXSCAN', 'keyPattern': {'city': 1}, 'indexName': 'city_1', 'isMultiKey': False, 'multiKeyPaths': {'city': []}, 'isUnique': False, 'isSparse': False, 'isPartial': False, 'indexVersion': 2, 'direction': 'forward', 'indexBounds': {'city': ['["Santa Barbara", "Santa Barbara"]']}}}, 'rejectedPlans': []}, 'executionStats': {'executionSuccess': True, 'nReturned': 0, 'executionTimeMillis': 0, 'totalKeysExamined': 0, 'totalDocsExamined': 0, 'executionStages': {'stage': 'FETCH', 'nReturned': 0, 'executionTimeMillisEstimate': 0, 'works': 1, 'advanced': 0, 'needTime': 0, 'needYield': 0, 'saveState': 0, 'restoreState': 0, 'isEOF': 1, 'docsExamined': 0, 'al

In [41]:
print(ris)

{'explainVersion': '1', 'queryPlanner': {'namespace': 'yelpPython.collection_business_restaurants', 'indexFilterSet': False, 'parsedQuery': {'city': {'$eq': 'Santa Barbara'}}, 'maxIndexedOrSolutionsReached': False, 'maxIndexedAndSolutionsReached': False, 'maxScansToExplodeReached': False, 'winningPlan': {'stage': 'COLLSCAN', 'filter': {'city': {'$eq': 'Santa Barbara'}}, 'direction': 'forward'}, 'rejectedPlans': []}, 'executionStats': {'executionSuccess': True, 'nReturned': 0, 'executionTimeMillis': 0, 'totalKeysExamined': 0, 'totalDocsExamined': 0, 'executionStages': {'stage': 'COLLSCAN', 'filter': {'city': {'$eq': 'Santa Barbara'}}, 'nReturned': 0, 'executionTimeMillisEstimate': 0, 'works': 2, 'advanced': 0, 'needTime': 1, 'needYield': 0, 'saveState': 0, 'restoreState': 0, 'isEOF': 1, 'direction': 'forward', 'docsExamined': 0}, 'allPlansExecution': []}, 'command': {'find': 'collection_business_restaurants', 'filter': {'city': 'Santa Barbara'}, '$db': 'yelpPython'}, 'serverInfo': {'hos

### Compound index

In [ ]:
from pymongo import ASCENDING, DESCENDING

# compound index
db.collection_business_restaurants.create_index([("city", ASCENDING)], 
                       name="city_index",
                       background=True,
                       unique=False,
                       sparse=False)

# print out available indices
print(list(db.collection_business_restaurants.list_indexes()))

### Text index

In [33]:
# text index
db.tweets.create_index([('source', pymongo.TEXT)], default_language='english',name='text_index')
print(list(db.tweets.list_indexes()))

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')]), SON([('v', 2), ('key', SON([('created_at', 1), ('retweet_count', -1)])), ('name', 'twoLevel_index'), ('background', True), ('sparse', False)]), SON([('v', 2), ('key', SON([('_fts', 'text'), ('_ftsx', 1)])), ('name', 'text_index'), ('weights', SON([('source', 1)])), ('default_language', 'english'), ('language_override', 'language'), ('textIndexVersion', 3)])]


In [34]:
filter = {
    "$text": {"$search": "iphone"}
}

# proiezione di dati
project={
    'source': 1
}

result = db.tweets.find(
  filter=filter,
  projection=project
)

result_list = list(result)
len(result_list),result_list[0]

(16,
 {'_id': ObjectId('6265887a51d11a414a387dae'),
  'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'})